# DATA CARD

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [20]:
df = pd.read_sql_query("select w.name warehouse, oc.name operation, count(distinct(t.id)) quantity\
    from datex_footprint.Tasks t\
        join datex_footprint.OperationCodes oc\
            on oc.id = t.operationCodeId\
        join datex_footprint.Warehouses w\
            on w.id = t.warehouseId\
    where t.createdSysDateTime > '2022-04-20' AND oc.name in ('Batch Move', 'Count', 'Inventory Adjustment', 'InventoryMove', 'License Plate Pick', 'LicensePlateMove', 'Picking', 'Receiving')\
    group by w.name, oc.name\
    order by warehouse, operation", cnxn)

# Create Data Card

In [21]:
df.head()

,warehouse,operation,quantity
0,10,Batch Move,52
1,10,Count,2
2,10,Inventory Adjustment,2
3,10,InventoryMove,4
4,10,License Plate Pick,28


In [22]:
# Convert Statuses to Columns
dc_df = df.pivot(index='warehouse', columns='operation', values='quantity')
dc_df

operation,Batch Move,Count,Inventory Adjustment,InventoryMove,License Plate Pick,LicensePlateMove,Picking,Receiving
warehouse,,,,,,,,
10,52.0,2.0,2.0,4.0,28.0,32.0,576.0,40.0
12,NaN,NaN,NaN,NaN,NaN,1.0,132.0,NaN
15,NaN,8.0,NaN,NaN,NaN,1.0,55.0,2.0
16,NaN,NaN,NaN,NaN,21.0,17.0,59.0,24.0
18,8.0,NaN,NaN,2.0,NaN,3.0,142.0,16.0
19,NaN,NaN,1.0,9.0,18.0,30.0,93.0,NaN
20,NaN,45.0,1.0,1.0,14.0,13.0,186.0,4.0
21,NaN,NaN,NaN,NaN,59.0,43.0,62.0,23.0
22,NaN,NaN,NaN,NaN,13.0,1.0,42.0,31.0


In [28]:
# Convert NaN to 0
dc_df_zero = dc_df.fillna(0)
# Convert Float to Int
df2 = dc_df_zero.astype(int)
# Replace Zero with Empty
df2.replace([0, ''], '', inplace=True)

df2

operation,Batch Move,Count,Inventory Adjustment,InventoryMove,License Plate Pick,LicensePlateMove,Picking,Receiving
warehouse,,,,,,,,
10,52,2,2,4,28,32,576,40
12,,,,,,1,132,
15,,8,,,,1,55,2
16,,,,,21,17,59,24
18,8,,,2,,3,142,16
19,,,1,9,18,30,93,
20,,45,1,1,14,13,186,4
21,,,,,59,43,62,23
22,,,,,13,1,42,31


In [29]:
# Save Waves DF into json file
df2.to_json('data/card.json')

In [30]:
# Close connection
cnxn.close()